In [1]:
import numpy as np
import scipy as scy
import matplotlib.pyplot as plt

In [2]:
def inicializa_sistema():
    L = 20
    x_min = -L/2
    x_max = L/2
    y_min = -L/2
    y_max = L/2
    
    tam_carga = 0.3
    n_linhas = 10
    dt = 0.4
    #dtheta = 2*np.pi/n_linhas
    n_frames = 15
    for i in range(n_frames):
        Desenha_Campo(L, i, n_frames, tam_carga, dt)

In [3]:
def zero_of_f(f,dfdt,R, t, t_ret, ta, tb):
    eps = 10**(-6)
    dt_r = tb-ta
    for i in range(100):
        dt = f/dfdt
        sinal = (t_ret - ta -dt)*(t_ret - tb - dt)
        dt_old = dt
        if sinal>=0 or np.abs(2*dt) > np.abs(dt_old):
            dt_r = 0.5*(ta-tb)
            t_ret = tb + dt_r
        else :
            dt_r = dt
            t_ret = t_ret - dt_r
        if np.abs(dt_r) < eps:
            return t_ret
        f,dfdt = fanddf(R,t,t_ret)
        if f < 0:
            tb = t_ret
        else:
            ta = t_ret
    return dt_r
            
def movimento(R,t_ret):
    r_ret = np.ones(3)
    v_ret = np.ones(3)
    a_ret = np.ones(3)
    
    ### posição retardada = posição atual
    r_ret[0] = R[0] - 0.2*np.cos(t_ret)
    r_ret[1] = R[1]
    r_ret[2] = R[2]
    
    ### velocidade retardada
    v_ret[0] = -0.2*np.sin(t_ret)
    v_ret[1] = 0 
    v_ret[2] = 0
    
    ### aceleração
    a_ret[0] = -0.2*np.cos(t_ret)
    a_ret[1] = 0
    a_ret[2] = 0
    
    return r_ret, v_ret, a_ret

In [4]:
def fanddf(R, t, t_ret):
    r_ret,v_ret,a_ret = movimento(R,t_ret)
    dist_ret = np.sqrt(np.dot(r_ret,r_ret))
    f = t - t_ret - dist_ret
    dfdt = -1 + np.dot(r_ret,v_ret)/dist_ret
    return f, dfdt

In [5]:
def retarted_time(R,t):
    tb = t
    fb, dfdt = fanddf(R ,t ,tb )
    ta = -1/(1.6)
    fa, dfdt = fanddf(R, t, ta)
    while(fa < 0):
        ta = ta*1.6
        fa, dfdt = fanddf(R, t, ta)
    t_ret = 0.5*(ta+tb)
    f, dfdt = fanddf(R,t,t_ret)
    return zero_of_f(f,dfdt,R,t,t_ret,ta,tb)    

In [6]:
def Calcula_Campo(R,t):
    
    t_ret = retarted_time(R,t)
    r_ret,v_ret,a_ret = movimento(R, t_ret)
    
    v2 = np.dot(v_ret,v_ret)
    dist_ret = np.sqrt(np.dot(r_ret,r_ret))
    
    u_ret = r_ret/dist_ret - v_ret
    uxa = np.cross(u_ret,a_ret)
    w_ret = np.cross(r_ret,uxa)
    ru = np.dot(r_ret,u_ret)
    E0 = dist_ret/(ru**3)
    
    return E0*(u_ret*(1-v2) + w_ret)

In [7]:
def Desenha_Campo(L, frame, n_frames, a, dt):
    linhas_x = []
    linhas_y = []
    E = np.zeros(3)
    R = np.zeros(3)
    ds = 1
    t = frame*dt
    r_ret, v_ret, a_ret = movimento(R, t)
    x = R[0] - r_ret[0]
    y = R[1] - r_ret[1]
    vec_dtheta = np.linspace(0,2*np.pi,n_frames)
    for theta in vec_dtheta:
        x_line = x + a*np.cos(theta)
        y_line = y + a*np.sin(theta)
        linhas_x.append(x_line)
        linhas_y.append(y_line)
        while(np.abs(x_line) < L/2 or np.abs(y_line) < L/2):
            R[0] = x_line
            R[1] = y_line
            E = Calcula_Campo(R,t)
            E_mag = np.sqrt(np.dot(E,E))
            x_line = x_line + ds*E[0]/E_mag
            y_line = y_line + ds*E[1]/E_mag
            linhas_x.append(x_line)
            linhas_y.append(y_line)
            plt.plot(x_line,y_line)
    print(len(linhas_x))
            
        

In [ ]:
inicializa_sistema()